<a href="https://colab.research.google.com/github/jlee335/cs470_Team7_VideoEnhancer/blob/main/Attempt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##<Attempt 1>
#####This code is composed with

1.   Basic settings
2.   Tensorboard Settings
3.   Define Network
4.   Training Phase
5.   Model test

#####You should fill the part surrounded by ##################.
##### Mostly it is path and model file, training settings.

# 1. Basic Settings

In [ ]:
#@title Mount google drive
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')
 
# Specify the directory path where `Attempt1.ipynb` exists.
# ex) root = '/gdrive/My Drive/Final_Project/'
###########################
root = '/gdrive/My Drive/cs470_Team7_VideoEnhancer/'
###########################

**Import Libraries**

In [ ]:
#@title Import libraries

import os
import numpy as np
import time
from pathlib import Path
from PIL import Image
 
import matplotlib.pyplot as plt
 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import cv2 as cv
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.datasets import VOCSegmentation, SBDataset
from torchvision.datasets.vision import StandardTransform
from torchvision.utils import make_grid
from torchvision import transforms
from torchvision.models.vgg import VGG, vgg16, make_layers
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from math import log10
!pip install pydensecrf
from torch.autograd import Variable
from easydict import EasyDict as edict
 
import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils
 
!nvidia-smi
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100
 
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types


%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
#@title Setting parameters

torch.manual_seed(470)
torch.cuda.manual_seed(470)
 
args = edict()
 
args.batch_size = 1
args.lr = 1e-4
args.momentum = 0.9
args.weight_decay = 5e-4
args.epoch = 10
args.tensorboard = True
args.gpu = True

 
device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'
 

In [ ]:
#@title Set directory
# Put model file name in ckpt_path's end
#ex) ckpt_path = parent_dir / 'model.pt'
result_dir = Path(root) / 'data'
result_dir.mkdir(parents=True, exist_ok=True)

data_dir = result_dir / 'test_data'
data_dir.mkdir(parents=True, exist_ok=True)

num_trial=0
result_dir= Path(root) / 'results'
parent_dir = result_dir / f'trial_{num_trial}'
while parent_dir.is_dir():
    num_trial = int(parent_dir.name.replace('trial_',''))
    parent_dir = result_dir / f'trial_{num_trial+1}'
 
# Modify parent_dir here if you want to resume from a checkpoint, or to rename directory.
# parent_dir = result_dir / 'trial_99'

log_dir = parent_dir
ckpt_dir = parent_dir
#################################
ckpt_path = parent_dir / ''
#################################
writer = SummaryWriter(log_dir)
 

In [ ]:
#@title Helper functions

normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
unnormalize = transforms.Normalize(mean = [-2.118, -2.036, -1.804], std = [4.367, 4.464, 4.444])

def optical_flow_warp(image, image_optical_flow):
    b, _ , h, w = image.size()
    grid = np.meshgrid(range(w), range(h))
    grid = np.stack(grid, axis=-1).astype(np.float64)
    grid[:, :, 0] = grid[:, :, 0] * 2 / (w - 1) -1
    grid[:, :, 1] = grid[:, :, 1] * 2 / (h - 1) -1
    grid = grid.transpose(2, 0, 1)
    grid = np.tile(grid, (b, 1, 1, 1))
    grid = Variable(torch.Tensor(grid))
    if image_optical_flow.is_cuda == True:
        grid = grid.cuda()

    flow_0 = torch.unsqueeze(image_optical_flow[:, 0, :, :] * 31 / (w - 1), dim=1)
    flow_1 = torch.unsqueeze(image_optical_flow[:, 1, :, :] * 31 / (h - 1), dim=1)
    grid = grid + torch.cat((flow_0, flow_1),1)
    grid = grid.transpose(1, 2)
    grid = grid.transpose(3, 2)
    output = F.grid_sample(image, grid, padding_mode='border')
    return output

def space_to_depth(x, block_size):
    n, c, h, w = x.size()
    unfolded_x = torch.nn.functional.unfold(x, block_size, stride=block_size)
    return unfolded_x.view(n, c * block_size ** 2, h // block_size, w // block_size)

def crop(tensor,top,left,h,w): #works!
    return tensor[:,:,top:top+h,left:left+w]


# Tensorboard Settings


In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# 2. Define Networks

In [ ]:
#@title FlowNet

# https://github.com/amanchadha/FRVSR-GAN/blob/master/FRVSRGAN_Models.py
class ConvLeaky(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(ConvLeaky, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_dim, out_channels=out_dim,
                               kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=out_dim, out_channels=out_dim,
                               kernel_size=3, stride=1, padding=1)

    def forward(self, input):
        out = self.conv1(input)
        out = F.leaky_relu(out, 0.2)
        out = self.conv2(out)
        out = F.leaky_relu(out, 0.2)
        return out


class FNetBlock(nn.Module):
    def __init__(self, in_dim, out_dim, typ):
        super(FNetBlock, self).__init__()
        self.convleaky = ConvLeaky(in_dim, out_dim)
        if typ == "maxpool":
            self.final = lambda x: F.max_pool2d(x, kernel_size=2, ceil_mode=True)
        elif typ == "bilinear":
            self.final = lambda x: F.interpolate(x, scale_factor=2, mode="bilinear")
        else:
            raise Exception('Type does not match any of maxpool or bilinear')

    def forward(self, input):
        out = self.convleaky(input)
        out = self.final(out)
        return out

# https://arxiv.org/pdf/1801.04590.pdf
class FNet(nn.Module):
    def __init__(self, in_dim=9):
        super(FNet, self).__init__()
        self.encoder = nn.Sequential(
            FNetBlock(in_dim, 32, typ="maxpool",),
            FNetBlock(32, 64, typ="maxpool"),
            FNetBlock(64, 128, typ="maxpool")
        )
        self.decoder = nn.Sequential(
            FNetBlock(128, 256, typ="bilinear"),
            FNetBlock(256, 128, typ="bilinear"),
            FNetBlock(128, 64, typ="bilinear")
        )
        self.conv1 = nn.Conv2d(in_channels=64, out_channels=32,
                               kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=2,
                               kernel_size=3, padding=1)

    def forward(self, input):
        _, _ , tmp_x, tmp_y = input.size()
        output = self.encoder(input)
        output = self.decoder(output) 
        output = crop(output,0,0, tmp_x, tmp_y)
        output = self.conv1(output)
        output = F.leaky_relu(output, 0.2)
        output = self.conv2(output)
        output = torch.tanh(output)
        
        return output


In [ ]:
#@title Discriminator Net 
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.l_relu = nn.LeakyReLU()
    self.outer_conv = nn.Conv2d(3,64,3)

    self.conv_64c = nn.Conv2d(64,64,3,2)
    self.bn_64c = nn.BatchNorm2d(64)

    self.conv_128e = nn.Conv2d(64,128,3,1)
    self.bn_128e = nn.BatchNorm2d(128)

    self.conv_128c = nn.Conv2d(128,128,3,2)
    self.bn_128c = nn.BatchNorm2d(128)

    self.conv_256e = nn.Conv2d(128,256,3,1)
    self.bn_256e = nn.BatchNorm2d(256)

    self.conv_256c = nn.Conv2d(256,256,3,2)
    self.bn_256c = nn.BatchNorm2d(256)

    self.conv_512e = nn.Conv2d(256,512,3,1)
    self.bn_512e = nn.BatchNorm2d(512)

    self.conv_512c = nn.Conv2d(512,512,3,2)
    self.bn_512c = nn.BatchNorm2d(512)

    self.flatten = nn.AdaptiveAvgPool2d(1)
    self.lin1 = nn.Conv2d(512, 1024, kernel_size=1)
    self.lin2 = nn.Conv2d(1024, 1, kernel_size=1)
    self.sig = nn.Sigmoid()

  def forward(self,x):
    x = self.l_relu(self.outer_conv(x))
    x = self.l_relu(self.bn_64c(self.conv_64c(x)))
    x = self.l_relu(self.bn_128e(self.conv_128e(x)))
    x = self.l_relu(self.bn_128c(self.conv_128c(x)))
    x = self.l_relu(self.bn_256e(self.conv_256e(x)))
    x = self.l_relu(self.bn_256c(self.conv_256c(x)))
    x = self.l_relu(self.bn_512e(self.conv_512e(x)))
    x = self.l_relu(self.bn_512c(self.conv_512c(x)))

    x = self.flatten(x)
    x = self.l_relu(self.lin1(x))
    x = self.sig(self.lin2(x))

    return x

In [ ]:
#@title SRFlowNet

class Residual_Block(nn.Module):
    def __init__(self):
      super(Residual_Block, self).__init__()
      self.conv1 = nn.Conv2d(64,64,3,padding=1)
      self.conv2 = nn.Conv2d(64,64,3,padding=1)
      self.relu = nn.ReLU()

    def forward(self,x):
      rc = x
      x = self.conv1(x)
      x = self.relu(x)
      x = self.conv2(x)
     
      x = x + rc
      return x

class SRNet(nn.Module):
    def __init__(self):
      super(SRNet, self).__init__()
      self.conv_init = nn.Conv2d(3,64,3,padding = 1)
      self.relu = nn.ReLU()

      self.res_thru = nn.Sequential(
                Residual_Block(),
                Residual_Block(),
                Residual_Block(),
                Residual_Block(),
                Residual_Block(),
                Residual_Block(),
                Residual_Block(),
                Residual_Block(),
                Residual_Block(),
                Residual_Block()
              )
      self.ct1 = nn.ConvTranspose2d(64,64,3,2)

      self.conv_img = nn.Conv2d(64,3,3,padding = 1)

    def forward(self, x):
      x = x[1]
      shape = x.shape

      x = self.conv_init(x) 
      x = self.relu(x)

      residual = x

      x = self.res_thru(x)
      
      x += residual

      x = self.ct1(x)
      x = self.relu(x)

      x = self.conv_img(x)

      return crop(x,1,1,shape[2] * 2,shape[3] * 2)

class SRFlowNet(nn.Module):
  def __init__(self):
    super(SRFlowNet, self).__init__()
    self.sr_net = SRNet()
    self.conv_init = nn.Conv2d(6,64,3,padding = 1)
    self.relu = nn.ReLU()
    self.fnet = FNet()

    self.res_thru = nn.Sequential(
              Residual_Block(),
              Residual_Block(),
              Residual_Block(),
              Residual_Block(),
              Residual_Block(),
              Residual_Block(),
              Residual_Block(),
              Residual_Block(),
              Residual_Block(),
              Residual_Block()
            )
    self.ct1 = nn.ConvTranspose2d(64,64,3,2)
    self.conv_img = nn.Conv2d(64,3,3,padding = 1)

  def forward(self, input):
    _, x, _ = input
    f_x = torch.cat(input, dim=1)

    curr = x
    fs = self.fnet(f_x)
    shape = x.shape
    x_warp = optical_flow_warp(curr, fs)
    x = torch.cat((x, x_warp), dim=1)

    x = self.conv_init(x) 
    x = self.relu(x)
    residual = x
    x = self.res_thru(x)
    x += residual
    x = self.ct1(x)
    x = self.relu(x)
    x = self.conv_img(x)

    return crop(x,1,1,shape[2] * 2,shape[3] * 2)

In [ ]:
# https://github.com/dongheehand/SRGAN-PyTorch/blob/master/losses.py
class perceptual_loss(nn.Module):
    def __init__(self, vgg):
        super(perceptual_loss, self).__init__()
        self.vgg = vgg
        self.criterion = nn.MSELoss()
    def forward(self, HR, SR, layer = 'relu5_4'):       
        hr_feat = self.vgg(HR)
        sr_feat = self.vgg(SR)
        
        return self.criterion(hr_feat, sr_feat), hr_feat, sr_feat

Define directories for processed data...?

NVIDIA DALI dataloader pipeline configuration

In [ ]:
#@title DALI video pipline config
batch_size=3
sequence_length=3

initial_prefetch_size=16

video_directory = os.path.join(data_dir)
video_files = [video_directory + '/' + f for f in os.listdir(video_directory)]

shuffle=True
args.batch_size = batch_size

class VideoPipe(Pipeline):
    def __init__(self, batch_size, num_threads, device_id, data, shuffle):
        super(VideoPipe, self).__init__(batch_size, num_threads, device_id, seed=16)
        self.input = ops.VideoReader(device="gpu", filenames=data, sequence_length=sequence_length,
                                     shard_id=0, num_shards=1,
                                     random_shuffle=shuffle, initial_fill=initial_prefetch_size,pad_last_batch = True)

    def define_graph(self):
        output = self.input(name="Reader")
        return output


pipe = VideoPipe(batch_size=batch_size, num_threads=2, device_id=0, data=video_files, shuffle=shuffle)



# Training Phase

GAN training Code

In [ ]:
#@title GAN training code
 
 
def train_net_GAN(net,discriminator, resume,n_iter,n_iter_desc,gen_epoc,dis_epoch):
    pipe.build()
    criterion = nn.MSELoss().to(device) # L2

    randcrop = torch.nn.Sequential(
        torchvision.transforms.RandomCrop(400, padding=None, pad_if_needed=False, fill=0, padding_mode='constant'),
        )
    if resume:
        checkpoint = torch.load(ckpt_path)
        net.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch = checkpoint['epoch']
        best_valid_iou = checkpoint['best_valid_iou']
        print(f'Resume training from epoch {epoch+1}')
    else:
        epoch = 0

    optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, net.parameters()),lr=args.lr)
    net.train()
 
    while epoch < gen_epoch: 
        t1 = time.time()
        first = True
        loss_total = 0

        for i in range(n_iter):
            pipe_out = pipe.run() 
            pipe_out = pipe_out[0].as_cpu().as_array()
            pipe_out = randcrop(torch.from_numpy(pipe_out).permute(0,1,4,2,3))

            frame_list = torch.chunk(pipe_out,chunks = pipe_out.shape[1],dim=1) 

            prev_frame =  frame_list[0].squeeze(1).to(device).float()/255.0
            frame =       frame_list[1].squeeze(1).float()/255.0       
            next_frame =  frame_list[2].squeeze(1).to(device).float()/255.0
 
            h = frame.shape[2]
            w = frame.shape[3]
 
            prev_lq =   torch.nn.functional.interpolate(prev_frame,size=(h//2,w//2))
            lq_frame =  torch.nn.functional.interpolate(frame,size=(h//2,w//2))
            next_lq =   torch.nn.functional.interpolate(next_frame,size=(h//2,w//2))
 
            prev_frame =  normalize(prev_frame).to(device)
            frame     =   normalize(frame).to(device)
            next_frame =  normalize(next_frame).to(device)
 
            prev_lq =  normalize(prev_lq).to(device)
            lq_frame  =   normalize(lq_frame).to(device)
            next_lq =  normalize(next_lq).to(device)

            frame_recon = net((prev_lq,lq_frame,next_lq))
            loss = criterion(frame,frame_recon)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            writer.add_scalar("gen-only-loss/train", loss, epoch*n_iter + i)
 
            loss_total += loss.item()/n_iter
       
        t = time.time() - t1
        
        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(checkpoint, ckpt_path)
        epoch += 1
        writer.flush()

        print("--epoch finished-- " + str(epoch-1) + " time taken: " + str(t))
        print(f'loss : {loss_total:.5f}')

    vgg_net = torchvision.models.vgg19(pretrained=True).to(device)
    vgg_net.eval()
    VGG_loss = perceptual_loss(vgg_net)
    cross_ent = nn.BCELoss()
 
    real_label = torch.ones((batch_size, 1,1,1)).to(device)
    fake_label = torch.zeros((batch_size, 1,1,1)).to(device)
 
    ce_loss = nn.BCELoss()
 
    discriminator.train()
    disc_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, discriminator.parameters()),lr=args.lr)

    epoch = 0
    torch.save(net.state_dict(),  result_dir / f'Attempt_1.ckpt')

    print("TRAINING DISCRIMINATOR NETWORK")
    while epoch < dis_epoch: 
        t1 = time.time()
        first = True
        loss_total = 0
        L2_Loss = 0
        adv_Loss = 0
        disc_loss_total = 0

        for i in range(n_iter_desc):
            pipe_out = pipe.run()
            pipe_out = pipe_out[0].as_cpu().as_array()
            pipe_out = randcrop(torch.from_numpy(pipe_out).permute(0,1,4,2,3))
 
            frame_list = torch.chunk(pipe_out,chunks = pipe_out.shape[1],dim=1)
 
            prev_frame =  frame_list[0].squeeze(1).to(device).float()/255.0
            frame =       frame_list[1].squeeze(1).float()/255.0    
            next_frame =  frame_list[2].squeeze(1).to(device).float()/255.0
 
            h = frame.shape[2]
            w = frame.shape[3]
 
            prev_lq =   torch.nn.functional.interpolate(prev_frame,size=(h//2,w//2))
            lq_frame =  torch.nn.functional.interpolate(frame,size=(h//2,w//2))
            next_lq =   torch.nn.functional.interpolate(next_frame,size=(h//2,w//2))
 
            prev_frame =  normalize(prev_frame).to(device)
            frame     =   normalize(frame).to(device)
            next_frame =  normalize(next_frame).to(device)
 
            prev_lq =  normalize(prev_lq).to(device)
            lq_frame  =   normalize(lq_frame).to(device)
            next_lq =  normalize(next_lq).to(device)

            frame_recon = net((prev_lq,lq_frame,next_lq))

            fake_prob = discriminator(frame_recon)
            real_prob = discriminator(frame)
            
            fake_error = ce_loss(fake_prob,fake_label) 
            real_error = ce_loss(real_prob,real_label) 
 
            disc_loss = fake_error + real_error
            disc_optimizer.zero_grad()

            percep_loss, hr_feat, sr_feat = VGG_loss((frame + 1.0) / 2.0, (frame_recon + 1.0) / 2.0, layer = 'relu5_4') #re/scaling values to -1 ~ 1, 
            L2loss = criterion(frame,frame_recon)
 
            percep_loss *= 0.006 
            adversarial_loss = cross_ent(fake_prob, real_label) * 1e-3
 
            total_loss = percep_loss + adversarial_loss + L2loss
            optimizer.zero_grad()
 
 
            disc_loss.backward(retain_graph = True)
 
            total_loss.backward()
              
            disc_optimizer.step()
            optimizer.step()

            writer.add_scalar("adv-total_loss/train",       total_loss, epoch*n_iter_desc + i)
            writer.add_scalar("adv-adversarial_loss/train",  adversarial_loss, epoch*n_iter_desc + i)
            writer.add_scalar("adv-L2loss/train",  L2loss, epoch*n_iter_desc + i)
            writer.add_scalar("adv-disc_loss/train",  disc_loss, epoch*n_iter_desc + i)
 
            loss_total += total_loss.item()/n_iter
            adv_Loss += adversarial_loss.item()/n_iter
            L2_Loss += L2loss.item()/n_iter
            disc_loss_total += disc_loss.item()/n_iter

        t = time.time() - t1

        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(checkpoint, ckpt_path)
 
        epoch += 1

        writer.flush()

        print("--fine grain epoch finished-- " + str(epoch-1) + " time taken: " + str(t))
        print(f'loss : {loss_total:.5f} ,adv_loss :{adv_Loss:.5f},l2_loss: {L2_Loss:.5f} disc_loss :{disc_loss_total:.5f}')

    torch.save(net.state_dict(),  result_dir / f'Attempt_1.ckpt')
    torch.save(discriminator.state_dict(),  result_dir / f'disc_Attempt_1.ckpt')
    writer.close()

In [ ]:
#@title Set params and Train Model
 
#Set cont as True if you want to continue the training of gernerator based on existing ckpt
#load_disc is same as cont but for discriminator
#Put generator's file name and discriminator's file name in disc_file and gen_file.
#ex) disc_file = 'disc_Attempt_1.ckpt'
#ex) gen_file = 'Attempt_1.ckpt'

#Set number of epochs and iterations per epoch. ex) gen_epoch=5, gen_iter=100, dis_epoch = 12, dis_iter = 100

#Set cont as True if you want to continue the training of gernerator based on existing ckpt
#load_disc is same as cont but for discriminator

####################################
cont = False
load_disc = False
disc_file = ''
gen_file = ''
 
#Set number of epochs and iterations per epoch
gen_epoch = 
gen_iter = 
 
dis_epoch = 
dis_iter =
################################3 
 
if cont:

  model = SRFlowNet().to(device)
  desc = Discriminator().to(device)
 
 
  model.load_state_dict(torch.load(result_dir / gen_file))
  if load_disc:
    desc.load_state_dict(torch.load(result_dir / disc_file))
 
  train_net_GAN(model,desc,False,gen_iter,dis_iter,gen_epoch,dis_epoch)
 
else:
  model = SRFlowNet().to(device)
  desc = Discriminator().to(device)
 
  if load_disc:
    desc.load_state_dict(torch.load(result_dir / disc_file))
 
  train_net_GAN(model,desc,False,gen_iter,dis_iter,gen_epoch,dis_epoch)

# Model Test

In [ ]:
#@title Model Test For 3 Image as Input

#We checked the place that you should fill with ###########################
#imdir is the path for input images.
#Since we put 3 images, you should fill imdir1, imdir2, imdir3.

#svdir is path for saving result.
#ex) svdir = Path(root) / 'valid_test' / 'nalckiep.png'

###########################
model_file = 'Attempt_1.ckpt'
###########################
model = SRFlowNet().to(device)
model.load_state_dict(torch.load(result_dir / model_file))
model.eval()
 
images = []


loader = transforms.Compose([transforms.ToTensor()])
###########################
imdir1 = 
###########################
image1 = cv.imread(str(imdir1))
image1 = loader(image1).float()
image1 = image1.unsqueeze(0).to(device)
images.append(normalize(image1))
 
loader = transforms.Compose([transforms.ToTensor()])
###########################
imdir2 =
###########################
image2 = cv.imread(str(imdir2))
image2 = loader(image2).float()
image2 = image2.unsqueeze(0).to(device)
images.append(normalize(image2))
 
loader = transforms.Compose([transforms.ToTensor()])
###########################
imdir3 =
###########################
image3 = cv.imread(str(imdir3))
image3 = loader(image3).float()
image3 = image3.unsqueeze(0).to(device)
images.append(normalize(image3))
 

n_image = model(images)
n_image = unnormalize(n_image)
 
 
n_image2 = n_image.squeeze(0).transpose(0,2).transpose(0,1).cpu().detach().numpy()
n_image2 =(n_image2*255).astype(np.uint8)
###########################
svdir =
###########################
cv.imwrite(str(svdir),n_image2)

In [ ]:
#@title Model Test For Video as Input
#If you want to put video data as input, try this code below.

#Put video file as input in data.
#ex) data='/gdrive/My Drive/Final_Project/data/test_data/Video1.mp4'
#Put where to save your frames of your video.to "HERE"
#We hardly recommend you to MAKE "new foler" for frames.
#DO NOT change part after the path in name.
#ex)  name= '/gdrive/My Drive/Final_Project/data/test_data/frame/'+str(Number)+'.jpg'
###########################
video_filename=''
###################################

data = root + 'test/Before/' + video_filename
after = root + 'test/After/frames'
frames = root + 'test/Before/frames'

result = root + 'test/After/'


Path(after).mkdir(parents=True, exist_ok=True)
Path(frames).mkdir(parents=True, exist_ok=True)


vc=cv.VideoCapture(data)
Number=0

while(True):
  ret, frame=vc.read()
  if not (ret):
    break
  #####################################  
  name= frames + str(Number)+'.jpg'
  #########################################
  cv.imwrite(name,frame)
  Number+=1

vc.release()
cv.destroyAllWindows()


#Set model_file as .ckpt file
# ex) model_file = Path(root)/'Models'/'disc_IFNet_1129.ckpt'

#model_file ='/gdrive/My Drive/Final_Project/results/disc_IFNet_.ckpt'
#You should modify the path mentioned as "HERE" as same as upward.
#"THERE" is the path where you save the result frame.
#If you don't mine original picture overwritten, you can just use same path for "HERE" and "THERE".
#The path in "HERE", "THERE" SHOULD BE SAME!!!!
#ex)change "HERE" as '/gdrive/My Drive/Final_Project/data/test_data/frame'
#DO NOT modify part after "HERE" and "THERE"
#Modify (size) as the size you want.
#(size) SHOULD BE SAME and format of (x,y) and each x and y SHOULD BE DIVIDED TO 8!!!!!
#ex) (800,800)

##########################
model_file ='Attempt_1.ckpt'
##################################
model = IFNet().to(device)
model.load_state_dict(torch.load(result_dir / model_file))
model.eval()

frame_list=os.listdir(frames)

# Images need to be multiple of 16 to operate in our network.
image = cv.imread(frames + str(0)+'.jpg')
shape = image.shape
w_left = shape[0] % 16
h_left = shape[1] % 16
w_new = w - w_left
h_new = h - h_left
size = (w_new,h_new)

for i in range(1,len(frame_list)-1):
  images=[]
  
  image1= cv.imread(frames + str(i-1)+'.jpg')
  image1=cv.resize(image1,(size) )
  image2= cv.imread(frames + str(i)+'.jpg')
  image2=cv.resize(image2,(size) )
  image3= cv.imread(frames + str(i+1)+'.jpg')
  image3=cv.resize(image3,(size) )
  
  image1_=image1
  image2_=image2
  image3_=image3

  loader = transforms.Compose([transforms.ToTensor()])
  image1 = loader(image1).float()
  image1 = image1.unsqueeze(0).to(device)
  images.append(normalize(image1))

  loader = transforms.Compose([transforms.ToTensor()])
  image2 = loader(image2).float()
  image2 = image2.unsqueeze(0).to(device)
  images.append(normalize(image2))

  loader = transforms.Compose([transforms.ToTensor()])
  image3 = loader(image3).float()
  image3 = image3.unsqueeze(0).to(device)
  images.append(normalize(image3))

  n_image = model(images)
  n_image = unnormalize(n_image)
 
  n_image2 = n_image.squeeze(0).transpose(0,2).transpose(0,1).cpu().detach().numpy()
  n_image2 =(n_image2*255).astype(np.uint8)

  svname= after + str(i)+'.jpg'
  cv.imwrite(svname,n_image2)


#in VideoWriter, you should put code where to save your result video file.
#ex) video=cv2.VideoWriter('/gdrive/My Drive/VideoCapture/capture/Hi.avi',fourcc,15, (800,600))

#fourccis avi file's codec. Divx, Xvid, H.264 is available if you set your video file as .avi.
#If you want your output as .mp4 change fourcc 'XVID' as'mp4v' and change your video file name as .mp4.
#You can change FPS of video as you want.

#########################
fourcc = cv.VideoWriter_fourcc(*'XVID')
FPS= 15 # Default
video=cv.VideoWriter( result + '/Result.avi',fourcc,FPS, size)

n=os.listdir(after)
###################################


n = sorted(n)

for i in range (len(n)):
  img=cv.imread(after+n[i])
  video.write(img)

video.release()
cv.destroyAllWindows()
